In [3]:
# Importing useful packages
import numpy as np
from scipy import stats
import pandas as pd
import sklearn as sk
import seaborn as sb
import datetime as dt
import pylab 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from new_var import calc,y1function,y2function,C3function,C6function
from outliers import outlier

%matplotlib inline
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.cross_validation import train_test_split

ipo_data = pd.read_excel("Competition #1 Raw Data_UPDATED I3.xlsx",header=0,na_values='-' )
# Replacing NaN values with their corresponding mean value

ipo_data=ipo_data.fillna(ipo_data.mean())


# Removing 0 and negatives for T' calculations        
ipo_data['T2']=ipo_data['T2'].mask(ipo_data['T2'] <= 0,ipo_data['T2'].mean())
ipo_data['T1']=ipo_data['T1'].mask(ipo_data['T1'] <= 0,ipo_data['T1'].mean())
ipo_data['T3']=ipo_data['T3'].mask(ipo_data['T3'] <= 0,ipo_data['T3'].mean())
ipo_data['T4']=ipo_data['T4'].mask(ipo_data['T4'] <= 0,ipo_data['T4'].mean())
ipo_data['T5']=ipo_data['T5'].mask(ipo_data['T5'] <= 0,ipo_data['T5'].mean())
ipo_data['S1']=ipo_data['S1'].mask(ipo_data['S1'] <= 0,ipo_data['S1'].mean())
ipo_data.describe()


ImportError: cannot import name 'y1function'

In [ ]:
ipo_data['Y1'] = ipo_data.apply(y1function, axis=1)
ipo_data['Y2'] = ipo_data.apply(y2function, axis=1)
ipo_data['C3x'] = ipo_data.apply(C3function, axis=1)
ipo_data['C6x'] = ipo_data.apply(C6function, axis=1)
calc(ipo_data)
del ipo_data['C3']
del ipo_data['C5']
del ipo_data['C6']
del ipo_data['T1']
del ipo_data['T2']
del ipo_data['T3']
del ipo_data['T4']
del ipo_data['T5']
del ipo_data['S1']
del ipo_data['S2']
del ipo_data['S3']
ipo_data.describe()


In [ ]:
# Changing variables to correct type
ipo_data.C6x = ipo_data.C6x.astype(float)

ipo_data.C2 = ipo_data.C2.astype(int)
print(ipo_data.dtypes)


In [ ]:
sqrt_transform=['C6x','T3x','T5x','S1x','S2x','S3x'] #Variables that need to be square rooted
ipo_data[sqrt_transform]=ipo_data[sqrt_transform]**0.5 #square rooting variable

log_transform=['C1','C7'] #Variables to be log transformed
ipo_data[log_transform]=np.log(ipo_data[log_transform])#log transformation


def standard(data,method):
    if method == 1:
        X_std = (data - data.min(axis=0)) / (data.max(axis=0) - data.min(axis=0))
        data = X_std * (1 - 0) + 0
        return data
    elif method==2:
        data = (data)/(10**len(str(int(max(data))))) 
train=['C3x','C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2']
#normal_col=['C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x']
from sklearn import preprocessing


In [ ]:
# Looping through our functions and building models for each combination in order to view best results
for j in range(1,5):
    # Creating data set for the model
    x=pd.DataFrame.copy(ipo_data)
    for i in range(len(x.columns)): 
        if x.iloc[:,i].dtype == float:
            outlier(x.iloc[:,i],j)
            for l in range(1,2):
                if x.iloc[:,i].dtype == float:   
                    x.iloc[:,i]=standard(x.iloc[:,i],1)
    print('Outlier Method {}; Standardise Method {}'.format(j,l))
    
    # Model creation starts here
    logreg = LogisticRegression()
    
    train=['C3x','C4','C1','C7','T3x','T4x','T5x','S1x','S2x','S3x','C2']
    X=x[train]
    y=x['Y1']
    rfe = RFE(logreg, 18)
    rfe = rfe.fit(X,y)

    logit_model=sm.Logit(x['Y1'],x[train])
    result=logit_model.fit()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    y_pred = logreg.predict(X_test)

    kfold = model_selection.KFold(n_splits=10, random_state=7)
    modelCV = LogisticRegression()
    scoring = 'accuracy'
    results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
    #confusion_matrix = confusion_matrix(y_test, y_pred)

    print(rfe.support_)
    print(rfe.ranking_)
    print(result.summary())
    print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
    print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

    print(confusion_matrix)
    print(classification_report(y_test, y_pred))

    logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
    fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
    plt.figure()
    plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.savefig('Log_ROC')
    plt.show()

In [ ]:
# reducing our sample due to in balance between y2=1 and y2=0
df_subset = ipo_data.loc[ipo_data['Y2'] == 1].sample(300)
ipo_data = ipo_data.drop(df_subset.index)



for j in range(1,5):
    x=pd.DataFrame.copy(ipo_data)
    for i in range(len(x.columns)): 
        if x.iloc[:,i].dtype == float:
            outlier(x.iloc[:,i],j)             
            if x.iloc[:,i].dtype == float:   
                x.iloc[:,i]=standard(x.iloc[:,i],1)
    print('Outlier Method {}; Standardise Method {}'.format(j,1))
    
    logreg = LogisticRegression()
    #train=['C4','C6x','T4x','S2x']
    #train=['C3x','C4','T3x','T4x','S1x','S2x','C2']
    train=['C3x','C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2']
    X=x[train]
    y=x['Y2']
    rfe = RFE(logreg, 18)
    rfe = rfe.fit(X,y)

    logit_model=sm.Logit(x['Y2'],x[train])
    result=logit_model.fit()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    y_pred = logreg.predict(X_test)

    kfold = model_selection.KFold(n_splits=10, random_state=7)
    modelCV = LogisticRegression()
    scoring = 'accuracy'
    results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
    #confusion_matrix = confusion_matrix(y_test, y_pred)

    print(rfe.support_)
    print(rfe.ranking_)
    print(result.summary())
    print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
    print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

    print(confusion_matrix)
    print(classification_report(y_test, y_pred))

    logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
    fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
    plt.figure()
    plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.savefig('Log_ROC')
    plt.show()
